issues:
1.no manual testing
2.no remove stop word

In [35]:
#Importing the required Libraries
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import tensorflow as tf 
import re 
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import seaborn as sns 
plt.style.use('ggplot')
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.9.1


In [36]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [37]:
strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of accelerators:  1


In [38]:
fake_df = pd.read_csv('Fake.csv')
real_df = pd.read_csv('True.csv')

In [39]:
fake_df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [40]:
fake_df.isnull().sum()
real_df.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [41]:
fake_df.subject.unique()
real_df.subject.unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [42]:
fake_df.drop(['date', 'subject'], axis=1, inplace=True)
real_df.drop(['date', 'subject'], axis=1, inplace=True)

In [43]:
fake_df['class'] = 0 
real_df['class'] = 1

In [44]:
news_df = pd.concat([fake_df, real_df], ignore_index=True, sort=False)
news_df

,title,text,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0
...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",1
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,1
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [45]:
news_df['text'] = news_df['title'] + news_df['text']
news_df.drop('title', axis=1, inplace=True)

In [33]:
features = news_df['text']
targets = news_df['class']

X_train, x_test, y_train, y_test = train_test_split(features, targets, test_size=0.20, random_state=18)

In [48]:
x_test

32213    In circuitous speech, Trump touts VP pick Penc...
39740    Bomb attack kills one Bahraini policeman, woun...
40650    Seoul considers unilateral sanctions against N...
28194    House clears way for debate on Trump-backed he...
10034    About Time! Christian Group Sues Amazon and SP...
                               ...                        
6072      The PGA Just Gave Donald Trump A Big Middle F...
18028    MSNBC REPORTER SHOCKED After Leftists Call For...
5579      McConnell Just ADMITTED The NRA Must Approve ...
29761    Factbox: Trump meets with Boeing chief, others...
1148      NUTJOB Missouri State Rep Beheads Live Chicke...
Name: text, Length: 8980, dtype: object

In [47]:
y_test

32213    1
39740    1
40650    1
28194    1
10034    0
        ..
6072     0
18028    0
5579     0
29761    1
1148     0
Name: class, Length: 8980, dtype: int64

In [49]:
def normalize(data):
    normalized = []
    for i in data:
        i = i.lower()
        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', '', i)
        # get rid of non words and extra spaces
        i = re.sub('\\W', ' ', i)
        i = re.sub('\n', '', i)
        i = re.sub(' +', ' ', i)
        i = re.sub('^ ', '', i)
        i = re.sub(' $', '', i)
        normalized.append(i)
    return normalized

X_train = normalize(X_train)
x_test = normalize(x_test)

In [50]:
max_vocab = 10000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train)

In [51]:
X_train[0]

'time to go 84 yr old supreme court justice ginsberg tells college audience 3 term male u s senator is a female video justice ruth bader ginsburg erroneously labeled south carolina sen lindsey graham as one of the women of the senate on monday ginsburg s misstatement came in her remarks upon accepting an allegheny college award given to her and the late justice antonin scalia for civility in public life hours after justice neil gorsuch was sworn in to replace scalia ginsburg took to the podium in calling on lawmakers to work together but mistakenly identified south carolina s senior senator as a woman i thought back to the 1993 confirmation of my nomination to the court the hearing was altogether civil the vote was 96 to 3 for justice scalia the vote was unanimous ginsburg said let s hope members of congress the members that allegheny college has already honored vice president joe biden and senator john mccain the women of the senate senators dianne feinstein and lindsey graham let s h

In [52]:
# tokenize the text into vectors 
X_train = tokenizer.texts_to_sequences(X_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [53]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=256)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, padding='post', maxlen=256)

In [54]:
X_train[0]

array([  98,    2,  207, 7153, 5740,  256,  492,  132,  284, 1479,  841,
       1729,  321,  574, 2547,   35,    7,  244,   11,    4, 1560,  114,
        284, 8580, 6626, 5500,  298,  948, 1303, 3574, 1933,   18,   51,
          3,    1,  197,    3,    1,  155,    9,  227, 6626,    7,  362,
          6,   53,  953, 1406, 3810,   31,  841, 3077,  489,    2,   53,
          5,    1,  627,  284, 4233, 1936,   10,    6,  178,  436,  813,
         49,  284, 4079, 2377,   17, 3332,    6,    2, 1424, 1936, 6626,
        267,    2,    1, 6585,    6,  586,    9,  438,    2,  212,  691,
         34, 1659,  298,  948,    7,  428,  244,   18,    4,  431,   32,
        729,  140,    2,    1, 4218, 1696,    3,  179,  815,    2,    1,
        132,    1,  840,   17, 6181,  599,    1,  161,   17, 8444,    2,
        321,   10,  284, 1936,    1,  161,   17, 7533, 6626,   15,  379,
          7,  715,  258,    3,  201,    1,  258,    8,  841,   21,  313,
       5900,  727,   36, 1127, 1703,    5,  244,  3

In [55]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_vocab, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 32)               18560     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [56]:
y_train

18839    0
13614    0
37215    1
39032    1
10143    0
        ..
5294     0
1726     0
35653    1
25720    1
28947    1
Name: class, Length: 35918, dtype: int64

In [57]:
# We are going to use early stop, which stops when the validation loss no longer improve.
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=2,validation_split=0.1, batch_size=64, shuffle=True, callbacks=[early_stop])

Epoch 1/2
506/506 [==============================] - 56s 88ms/step - loss: 0.3756 - accuracy: 0.7884 - val_loss: 0.1058 - val_accuracy: 0.9691
Epoch 2/2
506/506 [==============================] - 46s 92ms/step - loss: 0.0782 - accuracy: 0.9788 - val_loss: 0.0479 - val_accuracy: 0.9852


In [58]:
model.save('ML_LSTM_v1.model')

INFO:tensorflow:Assets written to: ML_LSTM_v1.model\assets


INFO:tensorflow:Assets written to: ML_LSTM_v1.model\assets


In [59]:
model.evaluate(x_test, y_test)

281/281 [==============================] - 12s 41ms/step - loss: 0.0541 - accuracy: 0.9846


[0.054125070571899414, 0.9846324920654297]

In [60]:
pred = pd.DataFrame(model.predict(x_test))

binary_predictions = []

for i in pred:
    if i >= 0.5:
        binary_predictions.append(1)
    else:
        binary_predictions.append(0)

281/281 [==============================] - 12s 35ms/step


In [66]:
pred

,0
0,2.734938
1,4.883654
2,4.487127
3,4.678833
4,-3.173214
...,...
8975,-7.823924
8976,-7.112227
8977,-7.849168
8978,2.850576


In [65]:
pred = pd.DataFrame(model.predict(x_test))
#test_id = pd.DataFrame(x_test.index)

submission = pd.DataFrame(pred)
#submission.columns = ["id","label"]
submission.to_csv("ML_LSTM_v1_Submission.csv",index=False)

281/281 [==============================] - 9s 33ms/step
